<a href="https://colab.research.google.com/github/Phillsu/Colab_OpenVLA/blob/main/openVLA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install PyTorch with CUDA support in Colab (this uses the pip version)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

# Clone and install the openvla repo
!git clone https://github.com/openvla/openvla.git
%cd openvla
!pip install -e .

# Install Flash Attention 2 for training
# If any issues arise, you can clear the pip cache using `!pip cache purge`
!pip install packaging ninja
!ninja --version  # Verify Ninja is installed
!pip install "flash-attn==2.5.5" --no-build-isolation


Looking in indexes: https://download.pytorch.org/whl/cu124
Cloning into 'openvla'...
remote: Enumerating objects: 471, done.
remote: Counting objects: 100% (306/306), done.
remote: Compressing objects: 100% (162/162), done.
remote: Total 471 (delta 175), reused 155 (delta 143), pack-reused 165 (from 1)
Receiving objects: 100% (471/471), 248.67 KiB | 5.92 MiB/s, done.
Resolving deltas: 100% (239/239), done.
/content/openvla
Obtaining file:///content/openvla
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/moojink/dlimp_openvla to /tmp/pip-install-pxtag6sr/dlimp_128a38cee12c471d9c19554775995618
  Running command git clone --filter=blob:none --quiet https://github.com/moojink/dlimp_openvla /tmp/pip-install-pxtag6sr/dlimp_128a38cee12c471d9c19554775995618
  Resolved https://github.com/moojink/dlimp_openvla

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 5.9 MB/s eta 0:00:00
1.11.1.git.kitware.jobserver-1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.5.5-cp310-cp310-linux_x86_64.whl size=120592233 sha256=3fc5d8813904d32cfc77aa4b88d40169dbc12053edb6ee0f1d94159527d05ab0
  Stored in directory: /root/.cache/pip/wheels/b2/67/52/8b6d5fcffdd9e1ec868f554cdef8f03eedb4bf4dcac852fca2
Successfully built flash-attn


In [ ]:
!pip install -r https://raw.githubusercontent.com/openvla/openvla/main/requirements-min.txt

In [ ]:
!pip install -q -U flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 31.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import cv2
from PIL import Image

def get_from_camera():
    # 初始化攝像頭
    cap = cv2.VideoCapture(0)

    # 檢查攝像頭是否成功打開
    if not cap.isOpened():
        raise IOError("無法打開網路攝像頭")

    # 捕獲一幀
    ret, frame = cap.read()

    # 檢查是否成功捕獲
    if not ret:
        raise IOError("無法捕獲圖像")

    # 釋放攝像頭
    cap.release()

    # 將OpenCV的BGR格式轉換為RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 轉換為PIL Image對象
    pil_image = Image.fromarray(rgb_frame)

    return pil_image

In [ ]:
# Install minimal dependencies (`torch`, `transformers`, `timm`, `tokenizers`, ...)
# > pip install -r https://raw.githubusercontent.com/openvla/openvla/main/requirements-min.txt
from transformers import AutoModelForVision2Seq, AutoProcessor
from PIL import Image

import torch

# Load Processor & VLA
processor = AutoProcessor.from_pretrained("openvla/openvla-7b", trust_remote_code=True)
vla = AutoModelForVision2Seq.from_pretrained(
    "openvla/openvla-7b",
    attn_implementation="flash_attention_2",  # [Optional] Requires `flash_attn`
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).to("cuda:0")

# Grab image input & format prompt
image: Image.Image = get_from_camera()
prompt = "In: What action should the robot take to {<INSTRUCTION>}?\nOut:"

# Predict Action (7-DoF; un-normalize for BridgeData V2)
inputs = processor(prompt, image).to("cuda:0", dtype=torch.bfloat16)
action = vla.predict_action(**inputs, unnorm_key="bridge_orig", do_sample=False)
print(action)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 9.06 MiB is free. Process 27331 has 14.74 GiB memory in use. Of the allocated memory 14.58 GiB is allocated by PyTorch, and 61.77 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

'cuda:0'